# Mosaicking Rasters

This notebook demonstrates how to mosaic multiple raster files into a single raster using `rasterio` in Python. Mosaicking is useful in remote sensing to combine overlapping or adjacent images into a seamless dataset.

## Prerequisites
- Install required libraries: `rasterio`, `numpy`, `matplotlib` (listed in `requirements.txt`).
- At least two GeoTIFF files (e.g., `raster1.tif`, `raster2.tif`) with the same CRS and compatible bands. Replace file paths with your own data.

## Learning Objectives
- Merge multiple rasters into a single mosaic.
- Handle overlapping areas during mosaicking.
- Visualize and save the mosaicked raster.

In [ ]:
# Import required libraries
import rasterio
from rasterio.merge import merge
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Load Raster Files

Load multiple raster files to be mosaicked. Ensure they have the same CRS and number of bands.

In [ ]:
# Define paths to raster files
raster_paths = ['raster1.tif', 'raster2.tif']  # Replace with your raster files

# Open rasters and store in a list
raster_files = []
for path in raster_paths:
    src = rasterio.open(path)
    raster_files.append(src)

# Check CRS consistency
crs_list = [src.crs for src in raster_files]
if len(set(crs_list)) > 1:
    print('Warning: Rasters have different CRS. Reproject them first.')
else:
    print(f'All rasters have CRS: {crs_list[0]}')

# Check band count
band_counts = [src.count for src in raster_files]
if len(set(band_counts)) > 1:
    print('Warning: Rasters have different band counts.')
else:
    print(f'All rasters have {band_counts[0]} bands')

## Step 2: Mosaic the Rasters

Use `rasterio.merge` to combine the rasters into a single mosaic, handling overlaps with the 'first' method (uses the first raster’s value in overlapping areas).

In [ ]:
# Mosaic the rasters
mosaic_data, mosaic_transform = merge(raster_files, method='first')

# Create metadata for the mosaic
mosaic_meta = raster_files[0].meta.copy()
mosaic_meta.update({
    'height': mosaic_data.shape[1],
    'width': mosaic_data.shape[2],
    'transform': mosaic_transform
})

# Print mosaic information
print(f'Mosaic shape: {mosaic_data.shape}')

## Step 3: Visualize the Mosaic

Visualize the mosaicked raster, handling single-band or multi-band (RGB) cases.

In [ ]:
# Get bounds for plotting
with rasterio.open(raster_paths[0]) as src:
    bounds = src.bounds  # Approximate bounds for visualization

# Plot based on number of bands
plt.figure(figsize=(8, 8))
if mosaic_data.shape[0] == 1:
    # Single-band raster
    plt.imshow(mosaic_data[0], cmap='gray')
    plt.colorbar(label='Pixel Value')
    plt.title('Mosaicked Single-Band Raster')
else:
    # Multi-band raster (RGB)
    rgb = mosaic_data[:3].transpose(1, 2, 0)
    rgb = rgb / rgb.max() if rgb.max() > 0 else rgb  # Normalize for display
    plt.imshow(rgb)
    plt.title('Mosaicked RGB Composite')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 4: Save the Mosaicked Raster

Save the mosaicked raster to a new GeoTIFF file.

In [ ]:
# Save the mosaic
output_path = 'mosaic_raster.tif'
with rasterio.open(output_path, 'w', **mosaic_meta) as dst:
    dst.write(mosaic_data)

print(f'Mosaicked raster saved to: {output_path}')

## Step 5: Close Raster Files

Close all open raster files to free memory.

In [ ]:
# Close all raster files
for src in raster_files:
    src.close()

## Next Steps

- Replace `raster1.tif` and `raster2.tif` with your own raster files.
- Experiment with different merge methods (e.g., 'last', 'min', 'max').
- Ensure rasters have the same CRS and resolution before mosaicking.
- Proceed to the next notebook (`11_normalization_scaling.ipynb`) for normalization and scaling.

## Notes
- Rasters must have the same CRS and compatible bands for successful mosaicking.
- Overlapping areas are handled by the merge method (default: 'first').
- See `docs/installation.md` for troubleshooting library installation.